## Remote Setup

https://github.com/oceanprotocol/ocean.py/blob/main/READMEs/setup-remote.md

### Step 2: Setup-remote

In [2]:
#create evm accounts (one time)

from eth_account.account import Account
account1 = Account.create()
account2 = Account.create()

print(f"""
REMOTE_TEST_PRIVATE_KEY1={account1.key.hex()}, ADDRESS1={account1.address}
REMOTE_TEST_PRIVATE_KEY2={account2.key.hex()}, ADDRESS2={account2.address}
""")


REMOTE_TEST_PRIVATE_KEY1=0xb54acfac67bd484435d60af49804e062645f6f3c7cc67a334cd4f9e312b3bea6, ADDRESS1=0xAd4Cf90611ceD79cf71aDf1f5a61aA613F2199bA
REMOTE_TEST_PRIVATE_KEY2=0x5f7f108ad4898898ba5aa33131f4fcd6209411c5afe6b30fb97ec97151cd163b, ADDRESS2=0x3de39598f7FacEC0Ed95597809007d27da1DBed8



REMOTE_TEST_PRIVATE_KEY1=0xb54acfac67bd484435d60af49804e062645f6f3c7cc67a334cd4f9e312b3bea6, ADDRESS1=0xAd4Cf90611ceD79cf71aDf1f5a61aA613F2199bA
REMOTE_TEST_PRIVATE_KEY2=0x5f7f108ad4898898ba5aa33131f4fcd6209411c5afe6b30fb97ec97151cd163b, ADDRESS2=0x3de39598f7FacEC0Ed95597809007d27da1DBed8

### Step 5: Set envars

In [6]:
import os

# Setzen der Umgebungsvariablen
os.environ['REMOTE_TEST_PRIVATE_KEY1'] = '0xb54acfac67bd484435d60af49804e062645f6f3c7cc67a334cd4f9e312b3bea6'
os.environ['REMOTE_TEST_PRIVATE_KEY2'] = '0x5f7f108ad4898898ba5aa33131f4fcd6209411c5afe6b30fb97ec97151cd163b'
os.environ['MUMBAI_RPC_URL'] = 'wss://polygon-mumbai-bor.publicnode.com' # wenns nicht klappt einfach auf 'https://chainlist.org/?search=mumbai&testnets=true' einen anderen RPC-URL suchen

# Zugriff auf die Umgebungsvariablen
private_key1 = os.environ.get('REMOTE_TEST_PRIVATE_KEY1')
private_key2 = os.environ.get('REMOTE_TEST_PRIVATE_KEY2')
rpc_url = os.environ.get('MUMBAI_RPC_URL')

### Step 6: Set Up in Python

In [7]:
# Create Ocean instance
import os
from ocean_lib.example_config import get_config_dict
from ocean_lib.ocean.ocean import Ocean
config = get_config_dict(os.getenv("MUMBAI_RPC_URL"))  # you can also input the string directly
ocean = Ocean(config)

# Create OCEAN object. ocean_lib knows where OCEAN is on all remote networks
OCEAN = ocean.OCEAN_token

# Create Alice's wallet
from eth_account import Account

alice_private_key = os.getenv('REMOTE_TEST_PRIVATE_KEY1')
alice = Account.from_key(private_key=alice_private_key)
assert ocean.wallet_balance(alice) > 0, "Alice needs MATIC"
assert OCEAN.balanceOf(alice) > 0, "Alice needs OCEAN"

# Create Bob's wallet. While some flows just use Alice wallet, it's simpler to do all here.
bob_private_key = os.getenv('REMOTE_TEST_PRIVATE_KEY2')
bob = Account.from_key(private_key=bob_private_key)
assert ocean.wallet_balance(bob) > 0, "Bob needs MATIC"
assert OCEAN.balanceOf(bob) > 0, "Bob needs OCEAN"

# Compact wei <> eth conversion
from ocean_lib.ocean.util import to_wei, from_wei

## 2. Main Flow

In [8]:
# 1. Alice publishes a dataset (--> i.e an ocean asset)

#data info
name = "Branin dataset"
url = "https://raw.githubusercontent.com/trentmc/branin/main/branin.arff"

#create data asset
(data_nft, datatoken, ddo) = ocean.assets.create_url_asset(name, url, {"from": alice})

#print
print("Just published asset:")
print(f"  data_nft: symbol={data_nft.symbol}, address={data_nft.address}")
print(f"  datatoken: symbol={datatoken.symbol}, address={datatoken.address}")
print(f"  did={ddo.did}")

INFO:ocean_lib.data_provider.data_encryptor:Asset urls encrypted successfully, encrypted urls str: 0x040904fcb82af8a25e1e85b423d00f38986b0c3c4d8d3fec713d3f445ca3a6cb8efe59c3a4eb53c624a66a2a31028b30bf2de7a2b24d80b9848a5684f2688dfb7f68af62178a089df2f50440a9bd72b9fb3e99c6cb7be8baec2cbeab017cad950b296186c3e48455ffc7b4e710332424bfab38282be054fef11f93a9e41811fdb943b1c8d39c6b83b334b0f9529d6e58a0ab5757bb6a0a3c607846520f461960101deb547d8a93a9e9e7dfccaea48d31b69fae4271821d7169e2633a97ce73168b27e0f777b76b4fa0039c70a734154a657cbe8ec79d6be0e79dc86e6b1063d14d7bf72dad09ada70b46676ceffd6e0b89e2577e19f180a36c8c93d1c3d95588b28ba037d43d3faf73d9137e49b7ead93df85eba194e65b8fa1767b38a79b24fe87f1b61c63aa66d00ab558ce8e62189d096c8e5a964f23aaf9584208e5819, encryptedEndpoint https://v4.provider.mumbai.oceanprotocol.com/api/services/encrypt?chainId=80001
INFO:ocean_lib.data_provider.data_encryptor:Asset urls encrypted successfully, encrypted urls str: 0x0430b70fc86236b4813b57d9a051acdda567074d597ec35e9bb80015f09e

Just published asset:
  data_nft: symbol=<function function_wrapper.<locals>.wrap at 0x0000027002632840>, address=0x82a15D24f6302dA163A4a0124c9f140b74E12352
  datatoken: symbol=<function function_wrapper.<locals>.wrap at 0x0000027003790900>, address=0xc5BE3A999341d8EcC14d9761f1Bb52252Db9bEb1
  did=did:op:574f283d0efdc5b099c35e31c5839e6f4ac679c0760a9972b0949338c1832e8b


In [9]:
# 2. Bob gets access to the dataset

from ocean_lib.ocean.util import to_wei

#Approach A: Alice mints datatokens to Bob
datatoken.mint(bob, to_wei(1), {"from": alice})

#Approach B: Alice mints for herself, and transfers to Bob
datatoken.mint(alice, to_wei(1), {"from": alice})
datatoken.transfer(bob, to_wei(1), {"from": alice})

#Approach C: Alice posts for free, via a dispenser / faucet; Bob requests & gets
datatoken.create_dispenser({"from": alice})
datatoken.dispense(to_wei(1), {"from": bob})

#Approach D: Alice posts for sale; Bob buys
# D.1 Alice creates exchange
price = to_wei(100)
exchange = datatoken.create_exchange({"from": alice}, price, ocean.OCEAN_address)

# D.2 Alice makes 100 datatokens available on the exchange
datatoken.mint(alice, to_wei(100), {"from": alice})
datatoken.approve(exchange.address, to_wei(100), {"from": alice})

# D.3 Bob lets exchange pull the OCEAN needed
OCEAN_needed = exchange.BT_needed(to_wei(1), consume_market_fee=0)
ocean.OCEAN_token.approve(exchange.address, OCEAN_needed, {"from":bob})

# D.4 Bob buys datatoken
exchange.buy_DT(to_wei(1), consume_market_fee=0, tx_dict={"from": bob})


AttributeDict({'blockHash': HexBytes('0x2190010d723490885f3c668df0551c7dc90773bf6fc4b374cb68dd3503976a0d'),
 'blockNumber': 45681152,
 'contractAddress': None,
 'cumulativeGasUsed': 9129406,
 'effectiveGasPrice': 1500000015,
 'from': '0x3de39598f7FacEC0Ed95597809007d27da1DBed8',
 'gasUsed': 164620,
 'logs': [AttributeDict({'address': '0xd8992Ed72C445c35Cb4A2be468568Ed1079357c8',
   'blockHash': HexBytes('0x2190010d723490885f3c668df0551c7dc90773bf6fc4b374cb68dd3503976a0d'),
   'blockNumber': 45681152,
   'data': HexBytes('0x0000000000000000000000000000000000000000000000056d2aa3a5c09a0000'),
   'logIndex': 256,
   'removed': False,
   'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
    HexBytes('0x0000000000000000000000003de39598f7facec0ed95597809007d27da1dbed8'),
    HexBytes('0x00000000000000000000000025e1926e3d57ec0651e89c654ab0fa182c6d5cf7')],
   'transactionHash': HexBytes('0xcd4e1ac1f8d298557471cd95fe4a5315ca3eed74c4e75fab40de5835184c6670'

In [11]:
# 3. Bob consumes the dataset

# Bob sends a datatoken to the service to get access
order_tx_id = ocean.assets.pay_for_access_service(ddo, {"from": bob}).hex()

# Bob downloads the file. If the connection breaks, Bob can try again
asset_dir = ocean.assets.download_asset(ddo, bob, './', order_tx_id)

import os
file_name = os.path.join(asset_dir, "file0")

TypeError: typing.Any cannot be used with isinstance()

In [14]:
# Bob sends a datatoken to the service to get access
order_tx_id = ocean.assets.pay_for_access_service(ddo, {"from": bob}).hex()

TypeError: typing.Any cannot be used with isinstance()

In [13]:
import os
#lets check that the file is downloaded

# Change directory
os.chdir('my_project')

# Read the file
with open('datafile.did:op:*', 'r') as file:
    content = file.read()

print(content)
